In [25]:
!pip install pytesseract

In [26]:
!pip install Unidecode

In [27]:
import collections
!pip install tensorflow-text
import logging
import os
import pathlib
import re
import string
import sys
import time
# import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf
import pytesseract
from PIL import Image
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import io

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## loading and preprocessing dataset

In [29]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]
                  for l in lines[:num_examples]]
    return zip(*word_pairs)

In [30]:

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(
        tensor, padding='post')

    return tensor, lang_tokenizer

In [31]:
def load_dataset(path, num_examples=None):
    inp_lang, targ_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [32]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    w = re.sub(r"([?.!,|])", r" \1", w)
    w = re.sub(r'[" "]+', " ", w)
    w = w.strip()

    w = '<start> ' + w + ' <end>'
    return w
    
path_to_file = "./output.txt"
num_examples = 4332  # total no.of direct translation available to us
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(
    input_tensor, target_tensor, test_size=0.1)

#Plot Attention

In [33]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

#Loss Function

# Optimizer

In [34]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [35]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

#Predict.py

# Params defined

In [36]:


BUFFER_SIZE = len(input_tensor_train)
#@markdown > Add the Batch_size, By default it is 64.
BATCH_SIZE = 64  #@param{type:"number"}

steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
#@markdown > Add the Embedding dimension `embedding_dim`. By default it's 256.
embedding_dim = 256 #@param {type: "number" }
#@markdown > Add the units here. By default it's 1024.
units = 1024 #@param {type: "number" }

vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices(
    (input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Encoder class object

In [37]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [38]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# Decoder class object

In [39]:
import tensorflow as tf


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)

        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [40]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):

        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = self.embedding(x)

        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state = self.gru(x)

        output = tf.reshape(output, (-1, output.shape[2]))

        x = self.fc(output)

        return x, state, attention_weights

In [41]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

# Checkpoints

In [42]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


#Evaluate the model

In [43]:

def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

#Predict the model output

In [44]:
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    return result[:-6]


def input_data():
  input="What time does the shuttle bus leave for the airport"
  result=translate(input)
  return result

def input_data():
    from google.colab import files
    uploaded = files.upload()
    # img_path='/content/won.png'
    for n in uploaded.keys():
        img_path = '/content/test2.png'.format(n)
    im = Image.open(img_path)
    display(im)
    info = pytesseract.image_to_string(Image.open(img_path))
    print("\n\n")
    print("Text from image: ", info)
    try:
        print("Translated text: ", translate("I am dancing"))
    except:
        print("Can't translate.")

# Train Step

In [45]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims(
            [targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(
                dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

#Training and Predicting the Model output

In [ ]:
import time

start = time.time()
# training model
#@markdown > The number of epoch run on  the dataset. By default it's 400.
EPOCHS = 20#@param {type: "number" }


for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
          print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch,batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix=checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# saving encoder and decoder weights
encoder.save_weights('./encoder', save_format='tf')
decoder.save_weights('./decoder', save_format='tf')

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# final call to translation model
input_data()
print(start - time.time())

Epoch 1 Batch 0 Loss 2.2390
Epoch 1 Loss 1.7071
Time taken for 1 epoch 68.19211983680725 sec

Epoch 2 Batch 0 Loss 1.5833
Epoch 2 Loss 1.4213
Time taken for 1 epoch 21.001067876815796 sec

Epoch 3 Batch 0 Loss 1.1950
Epoch 3 Loss 1.3270
Time taken for 1 epoch 20.405682802200317 sec

Epoch 4 Batch 0 Loss 1.2122
Epoch 4 Loss 1.2295
Time taken for 1 epoch 20.853620052337646 sec

Epoch 5 Batch 0 Loss 1.0073
Epoch 5 Loss 1.1133
Time taken for 1 epoch 20.279304265975952 sec

Epoch 6 Batch 0 Loss 0.9475
Epoch 6 Loss 1.0094
Time taken for 1 epoch 20.812812089920044 sec

Epoch 7 Batch 0 Loss 0.7372
Epoch 7 Loss 0.9001
Time taken for 1 epoch 20.29562520980835 sec

Epoch 8 Batch 0 Loss 0.7675
Epoch 8 Loss 0.7952
Time taken for 1 epoch 20.79918932914734 sec

Epoch 9 Batch 0 Loss 0.7006
Epoch 9 Loss 0.6936
Time taken for 1 epoch 20.3478262424469 sec

Epoch 10 Batch 0 Loss 0.6087
Epoch 10 Loss 0.6007
Time taken for 1 epoch 20.869571208953857 sec

Epoch 11 Batch 0 Loss 0.4482
Epoch 11 Loss 0.5127
Tim

What time does the shuttle bus leave for the airport?	=== শাটেল বাসটা এয়ারপোর্টের জন্য কটার সময় ছাড়ে?